<a href="https://colab.research.google.com/github/naokityokoyama/Deep_Learing/blob/main/Deep_Learning_Games_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
games = pd.read_csv('1.1 games.csv')

In [ ]:
games.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
games.isnull().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Developer          6623
Rating             6769
dtype: int64

In [ ]:
games = games.drop(['Other_Sales', 'Global_Sales', 'Developer'], axis=1)

In [ ]:
games = games.dropna(axis=0)

In [ ]:
games = games.loc[games['NA_Sales']>1]
games = games.loc[games['EU_Sales']>1]

In [ ]:
games['Name'].value_counts()

Grand Theft Auto V                 4
Call of Duty: Ghosts               3
The Elder Scrolls V: Skyrim        3
Assassin's Creed IV: Black Flag    3
Battlefield 4                      3
                                  ..
Halo 3                             1
The Witcher 3: Wild Hunt           1
Need for Speed: Hot Pursuit        1
L.A. Noire                         1
Wii Music                          1
Name: Name, Length: 223, dtype: int64

In [ ]:
name = games.Name
games = games.drop('Name', axis=1)



In [ ]:
games

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
2,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
6,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,89.0,65.0,8.5,431.0,E
7,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,58.0,41.0,6.6,129.0,E
...,...,...,...,...,...,...,...,...,...,...,...,...
568,PS,1998.0,Shooter,Electronic Arts,1.44,1.09,0.00,92.0,17.0,8.6,80.0,T
573,PS2,2003.0,Action,Ubisoft,1.15,1.11,0.00,89.0,24.0,7.7,102.0,T
591,Wii,2007.0,Sports,Nintendo,1.05,1.05,0.24,79.0,47.0,8,124.0,E10+
610,PS2,2001.0,Racing,Acclaim Entertainment,1.13,1.12,0.06,80.0,15.0,7.9,46.0,T


In [ ]:
X = games.drop(['NA_Sales'	,'EU_Sales',	'JP_Sales'], axis=1)
y_na = games.NA_Sales
y_eu = games.EU_Sales
y_jp = games.JP_Sales

In [ ]:
le = LabelEncoder()

In [ ]:
X['Platform'] = le.fit_transform(X['Platform'])

In [ ]:
X.iloc[:,0] = le.fit_transform(X.iloc[:,0])
X.iloc[:,2] = le.fit_transform(X.iloc[:,2])
X.iloc[:,3] = le.fit_transform(X.iloc[:,3])
X.iloc[:,8] = le.fit_transform(X.iloc[:,8])

In [ ]:
lista_le = [0,2,3,8]

In [ ]:
#one hot encoder

onehotencoder = ColumnTransformer(transformers=[('OneHot', 
                                                 OneHotEncoder(), lista_le)], remainder='passthrough')
X = onehotencoder.fit_transform(X).toarray()

In [ ]:
#rede neural -> (61 +3) /2 
camada_entrada = Input(shape=(61,))
camada_oculta1 = Dense(units=32, activation='sigmoid')(camada_entrada)  #ligando a camada_entrada
camada_oculta2 = Dense(units=32, activation='sigmoid')(camada_oculta1) 
camada_saida1 = Dense(units=1, activation='linear')(camada_oculta2)
camada_saida2 = Dense(units=1, activation='linear')(camada_oculta2)
camada_saida3 = Dense(units=1, activation='linear')(camada_oculta2)

regressor = Model(inputs = camada_entrada, 
                  outputs = [camada_saida1, camada_saida2, camada_saida3])

regressor.compile(optimizer='adam', loss='mse')
regressor.fit (X, [y_na, y_eu, y_jp], epochs=5000, batch_size=100)

A saída de streaming foi truncada nas últimas 5000 linhas.
Epoch 2501/5000
3/3 [==============================] - 0s 5ms/step - loss: 5.4383 - dense_30_loss: 2.9952 - dense_31_loss: 1.7431 - dense_32_loss: 0.7000
Epoch 2502/5000
3/3 [==============================] - 0s 6ms/step - loss: 6.6296 - dense_30_loss: 3.7976 - dense_31_loss: 2.1208 - dense_32_loss: 0.7113
Epoch 2503/5000
3/3 [==============================] - 0s 5ms/step - loss: 5.5770 - dense_30_loss: 3.0246 - dense_31_loss: 1.7778 - dense_32_loss: 0.7745
Epoch 2504/5000
3/3 [==============================] - 0s 6ms/step - loss: 5.2390 - dense_30_loss: 2.7840 - dense_31_loss: 1.7624 - dense_32_loss: 0.6926
Epoch 2505/5000
3/3 [==============================] - 0s 5ms/step - loss: 6.3459 - dense_30_loss: 3.6102 - dense_31_loss: 1.9901 - dense_32_loss: 0.7455
Epoch 2506/5000
3/3 [==============================] - 0s 6ms/step - loss: 6.4294 - dense_30_loss: 3.5741 - dense_31_loss: 2.0113 - dense_32_loss: 0.8440
Epoch 2507/5000
3

In [ ]:
y_pred_na, y_pred_eu, y_pred_jp = regressor.predict(X)

In [ ]:
y_pred_na

In [ ]:
pd.DataFrame([y_na.values, y_pred_na]).T

,0,1
0,41.36,[31.333454]
1,15.68,[17.10129]
2,15.61,[14.19792]
3,11.28,[10.0095625]
4,13.96,[6.314914]
...,...,...
253,1.44,[2.2209086]
254,1.15,[2.220882]
255,1.05,[2.2224422]
256,1.13,[2.2208967]
